In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna
from datetime import datetime, timedelta

from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM, Informer, NHITS, DLinear
from neuralforecast.losses.pytorch import RMSE
from neuralforecast.losses.pytorch import DistributionLoss
from pytorch_forecasting import MAE

from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('once')

os.environ['NIXTLA_ID_AS_COL'] = '1'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


def data_preprocessing():
    return 

def sample_data(date_start, date_end, data_split, n_observations=None):
		df = data_preprocessing()
    date_start = datetime.strptime(date_start, '%Y-%m-%d')
    date_end = datetime.strptime(date_end, '%Y-%m-%d') + timedelta(hours=24)
		filtered_df = df[(df.index >= date_start) & (df.index <= date_end)]

		if n_observations is not None:
				filtered_df = filtered_df.head(n_observations)

		return np.split(df.sample(frac=1, random_state=42), [int(data_split[0] * len(df)), train_end + int(data_split[1] * len(df))])

def save_tuning(model_name, trial):
	try:
		df_tuning = pd.read_csv('tuning.csv')
	except:
		df_tuning = pd.DataFrame(columns=['model', 'accuracy', 'params'])

	new_row = {'model': model_name, 'accuracy': trial.value, 'params': str(trial.params)}
	new_row_df = pd.DataFrame([new_row]).dropna(axis=1, how='all')
	df_tuning = pd.concat([df_tuning, new_row_df], ignore_index=True)
	df_tuning = df_tuning.sort_values(by=['model', 'accuracy', 'params'], ascending=True).reset_index(drop=True)
	df_tuning.to_csv('tuning.csv', index=False)

def objective_LSTM(trial, data_train, data_test, forecast_horizon):
    nf = NeuralForecast(
        models=[LSTM(h=forecast_horizon, input_size=-1, loss=RMSE(),
                     encoder_n_layers=trial.suggest_categorical(
                         'encoder_n_layers', [1, 2, 5, 10]),
                     encoder_hidden_size=trial.suggest_categorical(
                         'encoder_hidden_size', [100, 200, 300, 400]),
                     context_size=trial.suggest_categorical(
                         'context_size', [5, 10, 15, 20]),
                     decoder_hidden_size=trial.suggest_categorical(
                         'decoder_hidden_size', [100, 200, 300, 400]),
                     decoder_layers=trial.suggest_categorical(
                         'decoder_layers', [1, 2, 5, 10]),
                     max_steps=trial.suggest_categorical(
                         'max_steps', [200, 500, 1000, 3000]),
                     val_check_steps=trial.suggest_categorical(
                         'val_check_steps', [10, 20, 50, 100, 250, 500]),
                     batch_size=trial.suggest_categorical(
                         'batch_size', [16, 32, 64, 128]),
                     scaler_type=trial.suggest_categorical(
                         'scaler_type', ['standard', 'minmax', 'robust']),
                     )
                ],
        freq='H'
    )
    nf.fit(data_train)
    predictions = nf.predict(data_test)
    return root_mean_squared_error(data_test['y'], predictions['LSTM'])


if __name__ == '__main__':
		# constant in tunings
    trials = 100
    date_start = '2023-11-01'
    date_end = '2024-11-01'

    # configurable variables
		model_name = 'LSTM'
		n_observations = None # Specify number of observations in dataset, otherwise None
		forecast_horizon = 30

    # constant sampling
    data_split = [.6, .2, .2] # 60% train, 20% validate, 20% test
    data_train, data_val, data_test =  = sample_data(date_start, date_end, data_split, n_observations)

    # configurable function
    objective_function = objective_LSTM(trial, data_train, data_val, data_test, forecast_horizon)

    def safe_objective(trial):
        try:
            return objective_function
        except Exception as e:
            print(f"Failed trial: {e}. Skipped this trial.")
            return float('inf')

    # warnings.filterwarnings("ignore")
    study1 = optuna.create_study(direction='minimize')
    study1.optimize(safe_objective, n_trials=trials)

    trial = study1.best_trial

    # warnings.filterwarnings("default")

    if trial.value != float('inf'):
      save_tuning(config_name=f'{model_name}_{forecast_horizon}', trial)